<a href="https://colab.research.google.com/github/nephylum/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/DS9_assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [x] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [x] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [x] Do one-hot encoding of categorical features.
- [x] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html). Use the scaler's `fit_transform` method with the train set. Use the scaler's `transform` method with the test set.
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling
import numpy as np

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)
df['LAND_SQUARE_FEET'] = (
    df['LAND_SQUARE_FEET']
    .replace('########', np.NaN)
    .str.replace(',','')
    .astype(float)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [171]:
# Use a subset of the data where BUILDING_CLASS_CATEGORY == 
#'01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and 
# less than 2 million.

filtered_df = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']
filtered_df = filtered_df[filtered_df['SALE_PRICE'] >= 100000 ]
filtered_df = filtered_df[filtered_df['SALE_PRICE'] <= 2000000]
filtered_df['SALE_PRICE'].value_counts()

500000    48
550000    43
450000    40
525000    40
650000    36
          ..
371000     1
413000     1
672045     1
457000     1
248000     1
Name: SALE_PRICE, Length: 1002, dtype: int64

In [172]:
filtered_df['SALE_DATE'] = pd.to_datetime(filtered_df['SALE_DATE'], infer_datetime_format=True)
filtered_df['SALE_DATE'].dt.year.value_counts()

2019    3164
Name: SALE_DATE, dtype: int64

In [173]:
# filter jan, feb mar into train then apr in test
train = filtered_df[(filtered_df['SALE_DATE'].dt.month == 1) | (filtered_df['SALE_DATE'].dt.month == 2) | (filtered_df['SALE_DATE'].dt.month == 3)]
test = filtered_df[filtered_df['SALE_DATE'].dt.month == 4]

print(len(train), len(test))
#check for loss of data in filtering (lengths should add up)
print('Are train and test same length?', len(train) + len(test) == len(filtered_df))


2517 647
Are train and test same length? True


In [0]:
# check for high-cardinality features that should be nixed, as well as
# columns with redundant/useless data

# train.select_dtypes(exclude='number').describe().T.sort_values(by='unique')
# train.select_dtypes(include='number').describe() #used to explore data

# It looks like I should drop SALE_PRICE, ADDRESS, BUILDING_CLASS_CATEGORY,
# APARTMENT_NUMBER, EASE-MENT
target = 'SALE_PRICE'
todrop =['ADDRESS', 'BUILDING_CLASS_CATEGORY', 'APARTMENT_NUMBER', 'EASE-MENT',
         'SALE_DATE'] #can't convert to string! (holdover)
features = train.columns.drop([target] + todrop)

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]


In [175]:
# change objects to int or floats where possible
X_train['BOROUGH'] = X_train.BOROUGH.astype(int)
X_test['BOROUGH'] = X_test.BOROUGH.astype(int)

X_test['BOROUGH'].dtype

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


dtype('int64')

In [0]:
# X_train['LAND_SQUARE_FEET'].value_counts()
# X_train

In [0]:
# Do one-hot encoding of categorical features.
import category_encoders as ce 
encoder = ce.OneHotEncoder(use_cat_names=True)
#need to fix land square feet (will do later)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [178]:
# Do feature selection with SelectKBest
from sklearn.feature_selection import f_regression, SelectKBest

selector = SelectKBest(score_func=f_regression, k = 20)

X_train_select = selector.fit_transform(X_train, y_train)
X_test_select = selector.transform(X_test)

#check the shape of the selected data
X_train_select.shape, X_test_select.shape

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


((2517, 20), (647, 20))

In [179]:
# Do feature scaling
# Use the scaler's `fit_transform` method with the train set
# Use the scaler's `transform` method with the test set.
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)+1):
  print(k,' features')

  selector = SelectKBest(score_func=f_regression, k = k)
  X_train_selected = selector.fit_transform(X_train, y_train)
  X_test_selected = selector.transform(X_test)

  model = LinearRegression()
  model.fit(X_train_selected, y_train)

  y_pred = model.predict(X_test_selected)
  mae = mean_absolute_error(y_test, y_pred)
  print(f'Mean Absolute Error of Test data: ${mae:.2f}\n')

1  features


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in

Mean Absolute Error of Test data: $185788.22

2  features
Mean Absolute Error of Test data: $181871.46

3  features
Mean Absolute Error of Test data: $181602.59

4  features
Mean Absolute Error of Test data: $181602.59

5  features
Mean Absolute Error of Test data: $173029.44

6  features
Mean Absolute Error of Test data: $172032.86

7  features
Mean Absolute Error of Test data: $171741.59

8  features
Mean Absolute Error of Test data: $158078.68

9  features
Mean Absolute Error of Test data: $157899.15

10  features
Mean Absolute Error of Test data: $157893.43

11  features
Mean Absolute Error of Test data: $159205.47

12  features
Mean Absolute Error of Test data: $159316.49

13  features
Mean Absolute Error of Test data: $159331.12

14  features
Mean Absolute Error of Test data: $159128.19

15  features
Mean Absolute Error of Test data: $159128.19

16  features
Mean Absolute Error of Test data: $159039.40

17  features
Mean Absolute Error of Test data: $159039.40

18  features
Mean 

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in

Mean Absolute Error of Test data: $159202.32

20  features
Mean Absolute Error of Test data: $156965.85

21  features
Mean Absolute Error of Test data: $156965.85

22  features
Mean Absolute Error of Test data: $156965.85

23  features
Mean Absolute Error of Test data: $156965.85

24  features
Mean Absolute Error of Test data: $156722.06

25  features
Mean Absolute Error of Test data: $156614.33

26  features
Mean Absolute Error of Test data: $156553.62

27  features
Mean Absolute Error of Test data: $156514.65


/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  ret


28  features
Mean Absolute Error of Test data: $157448.16

29  features
Mean Absolute Error of Test data: $157187.92

30  features
Mean Absolute Error of Test data: $157109.60

31  features
Mean Absolute Error of Test data: $157084.95

32  features
Mean Absolute Error of Test data: $157077.69

33  features
Mean Absolute Error of Test data: $156641.88

34  features
Mean Absolute Error of Test data: $156415.75

35  features
Mean Absolute Error of Test data: $156433.37

36  features
Mean Absolute Error of Test data: $156365.13

37  features
Mean Absolute Error of Test data: $156365.13

38  features
Mean Absolute Error of Test data: $156399.26


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in


39  features
Mean Absolute Error of Test data: $156428.43

40  features
Mean Absolute Error of Test data: $156431.07

41  features
Mean Absolute Error of Test data: $156704.31

42  features
Mean Absolute Error of Test data: $156704.31

43  features
Mean Absolute Error of Test data: $156704.31



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in

In [0]:
# Fit a ridge regression model with multiple features.

In [0]:
# Get mean absolute error for the test set.